In [1]:
from apscheduler.schedulers.asyncio import AsyncIOScheduler
from telegram import Bot
from telegram.ext import Application, CommandHandler
import requests
import feedparser
import re
from datetime import datetime
import pytz
import asyncio

In [2]:
# Список городов с их кодами для RSS-ленты
CITIES = {
    "Москва (ВДНХ)": "27612",
    "Москва (Шереметьево)": "27514",
    "Белый": "27613",
    "Смоленск": "26781",
    "Великие Луки": "26477",
    "Торопец": "26479",
    "Тверь": "27402",
    "Старица": "26499",
    # Добавьте другие города по мере необходимости
}

In [ ]:
# Функция для получения прогноза погоды из RSS
def get_weather_from_rss(city_name, city_code):
    rss_url = f"https://meteoinfo.ru/rss/forecasts/index.php?s={city_code}"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
    }
    
    try:
        response = requests.get(rss_url, headers=headers)
        response.raise_for_status()  # Проверка на HTTP-ошибки
        
        # Принудительно парсим содержимое как XML
        feed = feedparser.parse(response.content)
        
        if feed.bozo:  # Проверка на ошибки при парсинге
            return f"Ошибка при парсинге RSS для города {city_name}: {feed.bozo_exception}"
        
        # Извлекаем время обновления прогноза из тега <source>
        update_time_formatted = "Время обновления не указано"
        for entry in feed.entries:
            # Извлекаем текст из поля title тега <source>
            source_title = ""
            if "source" in entry and isinstance(entry["source"], dict):
                source_title = entry["source"].get("title", "").strip()
            
            print("Source title content:", repr(source_title))  # Отладочная информация
            
            # Ищем строку с временем обновления
            if source_title:
                update_time_match = re.search(
                    r"прогноз\s+обновл[её]н\s+(\d{2}\.\d{2}\.\d{4} в \d{2}:\d{2})\(utc\)", 
                    source_title, 
                    re.IGNORECASE
                )
                if update_time_match:
                    # Преобразуем время в объект datetime
                    update_time_str = update_time_match.group(1)
                    update_time_utc = datetime.strptime(update_time_str, "%d.%m.%Y в %H:%M")
                    
                    # Указываем, что время в UTC
                    utc_zone = pytz.utc
                    update_time_utc = utc_zone.localize(update_time_utc)
                    
                    # Конвертируем в локальное время
                    local_zone = pytz.timezone("Europe/Moscow")  # Замените на ваш часовой пояс
                    update_time_local = update_time_utc.astimezone(local_zone)
                    
                    # Форматируем время в читаемый вид
                    update_time_formatted = update_time_local.strftime("%d.%m.%Y в %H:%M")
        
        # Название города из первого элемента
        weather_data = [f"*Прогноз погоды от Гидрометцентра России* 🌡️", f"🏠 *{city_name}*"]
        
        for entry in feed.entries:
            title = entry.get("title", "Заголовок отсутствует")
            description = entry.get("summary", "Описание отсутствует")  # Используем summary вместо description
            
            # Извлекаем только дату из заголовка
            date_match = re.search(r"\d{1,2} [а-я]+", title)
            date = date_match.group(0) if date_match else "Дата отсутствует"
            
            # Разбиваем описание на отдельные строки
            description_lines = description.split(". ")
            
            # Разделяем строку с давлением и осадками
            pressure_line = description_lines[3] if len(description_lines) > 3 else ""
            precipitation_match = re.search(r"Вероятность осадков (\d+%)", pressure_line)
            precipitation = precipitation_match.group(0) if precipitation_match else "Вероятность осадков отсутствует"
            pressure = pressure_line.replace(precipitation, "").strip()
            
            # Формируем сообщение с иконками
            weather_message = (
                f"🗓️ *{date}*\n"
                f"*{description_lines[0]}* 🔹\n"  # Описание погоды жирным
                f"🌡️ {description_lines[1]}\n"    # Температура
                f"🌬️ {description_lines[2]}\n"    # Ветер
                f"📊 {pressure}\n"                # Давление
                f"🌧️ {precipitation}"             # Осадки
            )
            weather_data.append(weather_message)
        
        # Добавляем дату обновления и ссылку "Подробнее"
        link = feed.entries[0].get("link", "Ссылка отсутствует")   # Берем ссылку из первого элемента
        weather_data.append(f"🔄 Прогноз обновлён {update_time_formatted}")
        weather_data.append(f"[Подробнее]({link})")
        
        return "\n\n".join(weather_data)
    except requests.RequestException as e:
        return f"Ошибка при запросе для города {city_name}: {e}"
    except Exception as e:
        return f"Неизвестная ошибка для города {city_name}: {e}"

In [4]:
# Функция для отправки прогноза погоды в Telegram
async def send_scheduled_weather():
    TELEGRAM_TOKEN = "7991239059:AAFtDMISLWwRbaFiIUZEpJsAqUhqRZS62fo"  # Замените на ваш токен
    TELEGRAM_CHAT_ID = "1042080417"           # Замените на ваш chat_id

    bot = Bot(token=TELEGRAM_TOKEN)
    
    # Отправляем прогноз для каждого города
    for city_name, city_code in CITIES.items():
        weather = get_weather_from_rss(city_name, city_code)
        await bot.send_message(chat_id=TELEGRAM_CHAT_ID, text=weather, parse_mode="Markdown")
        print(f"Прогноз для города {city_name} отправлен.")

In [5]:
# Функция для запуска планировщика
def start_scheduler():
    scheduler = AsyncIOScheduler()
    
    # Добавляем задачу на отправку прогноза каждый день в 7:30
    scheduler.add_job(send_scheduled_weather, 'cron', hour=22, minute=15)
    
    # Запускаем планировщик
    scheduler.start()
    print("Планировщик запущен. Прогноз будет отправляться ежедневно в 7:30.")

In [6]:
# Основная функция
async def main():
    # Настройки Telegram
    TELEGRAM_TOKEN = "7991239059:AAFtDMISLWwRbaFiIUZEpJsAqUhqRZS62fo"  # Замените на ваш токен
    
    # Создаем приложение
    application = Application.builder().token(TELEGRAM_TOKEN).build()
    
    # Регистрируем обработчики команд
    application.add_handler(CommandHandler("start", lambda update, context: update.message.reply_text("Бот запущен.")))
    
    # Запускаем планировщик
    start_scheduler()
    
    # Запускаем бота
    await application.initialize()
    await application.start()
    await application.updater.start_polling()

if __name__ == "__main__":
    # Убедитесь, что вы не вызываете asyncio.run() вручную
    import nest_asyncio
    nest_asyncio.apply()
    
    # Запускаем основной цикл
    asyncio.run(main())

Планировщик запущен. Прогноз будет отправляться ежедневно в 7:30.


Source title content: 'meteoinfo.ru: Москва (ВДНХ), прогноз обновлён 01.04.2025 в 15:16(UTC)'
Source title content: 'meteoinfo.ru: Москва (ВДНХ), прогноз обновлён 01.04.2025 в 15:16(UTC)'
Source title content: 'meteoinfo.ru: Москва (ВДНХ), прогноз обновлён 01.04.2025 в 15:16(UTC)'
Прогноз для города Москва (ВДНХ) отправлен.
Source title content: 'meteoinfo.ru: Шереметьево, прогноз обновлён 01.04.2025 в 15:16(UTC)'
Source title content: 'meteoinfo.ru: Шереметьево, прогноз обновлён 01.04.2025 в 15:16(UTC)'
Source title content: 'meteoinfo.ru: Шереметьево, прогноз обновлён 01.04.2025 в 15:16(UTC)'
Прогноз для города Москва (Шереметьево) отправлен.
Source title content: 'meteoinfo.ru: Домодедово, прогноз обновлён 01.04.2025 в 15:16(UTC)'
Source title content: 'meteoinfo.ru: Домодедово, прогноз обновлён 01.04.2025 в 15:16(UTC)'
Source title content: 'meteoinfo.ru: Домодедово, прогноз обновлён 01.04.2025 в 15:16(UTC)'
Прогноз для города Белый отправлен.
Source title content: 'meteoinfo.ru: С